In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
# загрузка данных
images = np.load('images.npy')
labels = np.load('labels.npy')
images_sub = np.load('images_sub.npy')

# нормализация данных
images = images / 255.0
images_sub = images_sub / 255.0

# перевод разметки в тензоры
labels = torch.tensor(labels, dtype=torch.long)

In [3]:
# создание класса подготовки данных к обучению
class AlphabetDataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  # Преобразование к (C, H, W)
        
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image

In [8]:
# аугментация тренеровочных данных
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(48, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=None, shear=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3))
])

# # аугментация тестовых данных данных
transform_test = transforms.Compose([
    transforms.Resize(64),  # 
    transforms.CenterCrop(48),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
# создание датасетов
train_dataset = AlphabetDataset(images, labels, transform=transform_train)
test_dataset = AlphabetDataset(images_sub, transform=transform_test)

# разделение датасета на тренировочную и валидационную выборки
train_size = int(0.82 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
# перевод вычислений на gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#Создание класса модели на основе densenet201

class DenseNet201(nn.Module):
    def __init__(self):
        super(DenseNet201, self).__init__()
        self.base_model = models.densenet201(pretrained=True)
        num_features = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Linear(num_features, 26)  # 26 - количество классов

    def forward(self, x):
        x = self.base_model(x)
        return x

In [7]:
# инициализация модели, выбор оптимизатора
model = DenseNet201().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0007, weight_decay=1e-4)

In [13]:
# обучение модели
def train(model, train_loader, criterion, optimizer, epoch, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    avg_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / total
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {accuracy:.2f}%')

In [14]:
# валидация модели
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = running_loss / len(val_loader)
    accuracy = 100. * correct / total
    print(f'Val_Loss: {avg_loss:.2f}, Accuracy: {accuracy:.2f}%')

In [15]:
# запуск процесса обучения модели
num_epochs = 15
for epoch in range(num_epochs):
    train(model, train_loader, criterion, optimizer, epoch, device)
    validate(model, val_loader, criterion, device)

Epoch: 0, Loss: 1.24, Accuracy: 62.55%
Val_Loss: 0.66, Accuracy: 80.42%
Epoch: 1, Loss: 0.44, Accuracy: 86.79%
Val_Loss: 0.37, Accuracy: 89.11%
Epoch: 2, Loss: 0.30, Accuracy: 90.93%
Val_Loss: 0.27, Accuracy: 91.47%
Epoch: 3, Loss: 0.23, Accuracy: 92.60%
Val_Loss: 0.28, Accuracy: 91.22%
Epoch: 4, Loss: 0.16, Accuracy: 95.13%
Val_Loss: 0.37, Accuracy: 90.14%
Epoch: 5, Loss: 0.15, Accuracy: 95.34%
Val_Loss: 0.31, Accuracy: 91.19%
Epoch: 6, Loss: 0.13, Accuracy: 95.90%
Val_Loss: 0.26, Accuracy: 92.31%
Epoch: 7, Loss: 0.13, Accuracy: 95.91%
Val_Loss: 0.22, Accuracy: 93.33%
Epoch: 8, Loss: 0.09, Accuracy: 97.12%
Val_Loss: 0.29, Accuracy: 91.56%
Epoch: 9, Loss: 0.08, Accuracy: 97.54%
Val_Loss: 0.27, Accuracy: 92.39%
Epoch: 10, Loss: 0.08, Accuracy: 97.53%
Val_Loss: 0.40, Accuracy: 89.86%
Epoch: 11, Loss: 0.07, Accuracy: 97.67%
Val_Loss: 0.23, Accuracy: 93.39%
Epoch: 12, Loss: 0.06, Accuracy: 98.21%
Val_Loss: 0.26, Accuracy: 92.28%
Epoch: 13, Loss: 0.07, Accuracy: 97.82%
Val_Loss: 0.24, Accur

In [16]:
# предсказание результатов для тестовой выборки
def predict(model, test_loader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

predictions = predict(model, test_loader, device)

# сохранение резултатов
submission = pd.DataFrame({'Id': np.arange(len(predictions)), 'Category': predictions})
submission.to_csv('submission.csv', index=False)